In [11]:
import operator
import os
from dotenv import load_dotenv
load_dotenv()
from pydantic import BaseModel, Field
from typing import TypedDict, Annotated, Sequence
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import BaseMessage
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq


In [12]:
## Getting Froq API key
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

EMBEDDING_MODEL = "all-MiniLM-L6-v2"
INFERENCE_MODEL = "gemma2-9b-it"

In [13]:
llm = ChatGroq(model=INFERENCE_MODEL, temperature=0.3)
# print('--------Creating the response from llm based on contexts--------')

embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
len(embeddings.embed_query('Hi'))

e:\AgenticAI KrishNaikAcademy\Classroom HandsOn\agentic_class_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


384

In [8]:
class TopicSelectionParser(BaseModel):
    Topic: str=Field(description="Selected Topic")
    Reasoning: str=Field(description="Reasoining behind topic selection")

In [9]:
parser = PydanticOutputParser(pydantic_object=TopicSelectionParser)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Topic": {"description": "Selected Topic", "title": "Topic", "type": "string"}, "Reasoning": {"description": "Reasoining behind topic selection", "title": "Reasoning", "type": "string"}}, "required": ["Topic", "Reasoning"]}\n```'

In [2]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [14]:
def function_supervisor(state:AgentState):
    print("-> Inside Supervisor Function <-")
    question = state["messages"][-1]
    print(f"Question: {question}")

    template = """
    Your task is to classify the user's query into one of the following categories: [Unrelated, Wildlife Photography, real-time info]
    only respond with the category name and nothing more than that.

    User Query: {question}
    {format_output}
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["question"],
        partial_variables={"format_output":parser.get_format_instructions()}
    )

    chain = prompt | llm | parser
    response = chain.invole(prompt)

    print("Parsed response", response)

    return {"messages": [response.Topic]}

    

def function_llm(state:AgentState):
    pass

def function_rag(state:AgentState):
    pass

def function_web_crawl(state:AgentState):
    pass

In [4]:
def router(state:AgentState):
    print("-> Inside Router <-")

    last_message = state["messages"][-1]
    print(f"last_message: {last_message}")

    if "___" in last_message.lower():
        return "RAG Call"
    elif "llm" in last_message.lower():
        return "LLM Call"
    else:
        return "Web Crawler"